### Вайлдберис

Парсим список ноутов из каталога + общую инфу (цена, id и т.п.)

In [2]:
import requests
import json
import pandas as pd


"""
Парсер wildberries по ссылке на каталог (указывать без фильтров)
Парсер не идеален, есть множество вариантов реализации, со своими идеями 
и предложениями обязательно пишите мне, либо в группу, ссылка ниже.
Подробное описание парсера Вайлдберриз можно почитать на сайте:
https://happypython.ru/2022/07/21/парсер-wildberries/
Ссылка на статью ВКонтакте: https://vk.com/@happython-parser-wildberries
По всем возникшим вопросам, можете писать в группу https://vk.com/happython
парсер wildberries по каталогам 2022, обновлен 19.10.2022 - на данное число работает исправно
"""


def get_catalogs_wb():
    """получение каталога вб"""
    url = 'https://www.wildberries.ru/webapi/menu/main-menu-ru-ru.json'
    headers = {'Accept': "*/*", 'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    response = requests.get(url, headers=headers)
    data = response.json()
    with open('wb_catalogs_data.json', 'w', encoding='UTF-8') as file:
        json.dump(data, file, indent=2, ensure_ascii=False)
        print(f'Данные сохранены в wb_catalogs_data_sample.json')
    data_list = []
    for d in data:
        try:
            for child in d['childs']:
                try:
                    category_name = child['name']
                    category_url = child['url']
                    shard = child['shard']
                    query = child['query']
                    data_list.append({
                        'category_name': category_name,
                        'category_url': category_url,
                        'shard': shard,
                        'query': query})
                except:
                    continue
                try:
                    for sub_child in child['childs']:
                        category_name = sub_child['name']
                        category_url = sub_child['url']
                        shard = sub_child['shard']
                        query = sub_child['query']
                        data_list.append({
                            'category_name': category_name,
                            'category_url': category_url,
                            'shard': shard,
                            'query': query})
                except:
                    # print(f'не имеет дочерних каталогов *{i["name"]}*')
                    continue
        except:
            # print(f'не имеет дочерних каталогов *{d["name"]}*')
            continue
    return data_list


def search_category_in_catalog(url, catalog_list):
    """пишем проверку пользовательской ссылки на наличии в каталоге"""
    try:
        for catalog in catalog_list:
            if catalog['category_url'] == url.split('https://www.wildberries.ru')[-1]:
                print(f'найдено совпадение: {catalog["category_name"]}')
                name_category = catalog['category_name']
                shard = catalog['shard']
                query = catalog['query']
                return name_category, shard, query
            else:
                # print('нет совпадения')
                pass
    except:
        print('Данный раздел не найден!')


def get_data_from_json(json_file):
    """извлекаем из json интересующие нас данные"""
    data_list = []
    for data in json_file['data']['products']:
        try:
            price = int(data["priceU"] / 100)
        except:
            price = 0
        data_list.append({
            'Наименование': data['name'],
            'id': data['id'],
            'Скидка': data['sale'],
            'Цена': price,
            'Цена со скидкой': int(data["salePriceU"] / 100),
            'Бренд': data['brand'],
            'id бренда': int(data['brandId']),
            'feedbacks': data['feedbacks'],
            'rating': data['rating'],
            'Ссылка': f'https://www.wildberries.ru/catalog/{data["id"]}/detail.aspx?targetUrl=BP'
        })
    return data_list


def get_content(shard, query, low_price=None, top_price=None):
    # вставляем ценовые рамки для уменьшения выдачи, вилбериес отдает только 100 страниц
    headers = {'Accept': "*/*", 'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    data_list = []
    for page in range(1, 101):
        print(f'Сбор позиций со страницы {page} из 100')
        # url = f'https://wbxcatalog-ru.wildberries.ru/{shard}' \
        #       f'/catalog?appType=1&curr=rub&dest=-1029256,-102269,-1278703,-1255563' \
        #       f'&{query}&lang=ru&locale=ru&sort=sale&page={page}' \
        #       f'&priceU={low_price * 100};{top_price * 100}'
        url = f'https://catalog.wb.ru/catalog/{shard}/catalog?appType=1&curr=rub&dest=-1075831,-77677,-398551,12358499' \
              f'&locale=ru&page={page}&priceU={low_price * 100};{top_price * 100}' \
              f'&reg=0&regions=64,83,4,38,80,33,70,82,86,30,69,1,48,22,66,31,40&sort=popular&spp=0&{query}'
        r = requests.get(url, headers=headers)
        data = r.json()
        print(f'Добавлено позиций: {len(get_data_from_json(data))}')
        if len(get_data_from_json(data)) > 0:
            data_list.extend(get_data_from_json(data))
        else:
            print(f'Сбор данных завершен.')
            break
    return data_list


def save_excel(data, filename):
    """сохранение результата в excel файл"""
    df = pd.DataFrame(data)
    writer = pd.ExcelWriter(f'{filename}.xlsx')
    df.to_excel(writer, 'data')
    writer.save()
    print(f'Все сохранено в {filename}.xlsx')


def parser(url, low_price, top_price):
    # получаем список каталогов
    catalog_list = get_catalogs_wb()
    try:
        # поиск введенной категории в общем каталоге
        name_category, shard, query = search_category_in_catalog(url=url, catalog_list=catalog_list)
        # сбор данных в найденном каталоге
        data_list = get_content(shard=shard, query=query, low_price=low_price, top_price=top_price)
        # сохранение найденных данных
        save_excel(data_list, f'{name_category}_from_{low_price}_to_{top_price}')
    except TypeError:
        print('Ошибка! Возможно не верно указан раздел. Удалите все доп фильтры с ссылки')
    except PermissionError:
        print('Ошибка! Вы забыли закрыть созданный ранее excel файл. Закройте и повторите попытку')


if __name__ == '__main__':
    """ссылку на каталог или подкаталог, указывать без фильтров (без ценовых, сортировки и тд.)"""
    # url = input('Введите ссылку на категорию для сбора: ')
    # low_price = int(input('Введите минимальную сумму товара: '))
    # top_price = int(input('Введите максимульную сумму товара: '))

    """данные для теста. собераем товар с раздела велосипеды в ценовой категории от 50тыс, до 100тыс"""
    url = 'https://www.wildberries.ru/catalog/elektronika/noutbuki-pereferiya/noutbuki-ultrabuki'
    low_price = 5_000
    top_price = 1_000_000

    parser(url, low_price, top_price)

Данные сохранены в wb_catalogs_data_sample.json
найдено совпадение: Ноутбуки
Сбор позиций со страницы 1 из 100
Добавлено позиций: 100
Сбор позиций со страницы 2 из 100
Добавлено позиций: 100
Сбор позиций со страницы 3 из 100
Добавлено позиций: 100
Сбор позиций со страницы 4 из 100
Добавлено позиций: 100
Сбор позиций со страницы 5 из 100
Добавлено позиций: 100
Сбор позиций со страницы 6 из 100
Добавлено позиций: 100
Сбор позиций со страницы 7 из 100
Добавлено позиций: 100
Сбор позиций со страницы 8 из 100
Добавлено позиций: 100
Сбор позиций со страницы 9 из 100
Добавлено позиций: 84
Сбор позиций со страницы 10 из 100
Добавлено позиций: 0
Сбор данных завершен.
Ошибка! Вы забыли закрыть созданный ранее excel файл. Закройте и повторите попытку


Теперь бежим по ссылкам на конкретные ноуты и стягиваем характеристики

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
import json
from csv import reader
from tqdm import tqdm
import re

In [29]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

Из экселя сделаем csv с единственным столбцом ссылок на ноутбуки ('urls.csv'), т.е. оставим последний столбец и удалим заголовок

In [30]:
results = {}

with open('urls.csv', 'r') as file:
    for line in tqdm(file.readlines()):
        try:
            url = line

            #инициализируем словари для временного храниения информаци
            seller = dict()
            parameters = dict()

            driver.get(url=url)

            #даем странице время прогрузиться
            time.sleep(2 + 2 * random.random())

            #объект bs4 для парсинга
            soup = BeautifulSoup(driver.page_source, 'lxml')

            #вытаскиваем параметры из таблицы "О товаре"
            product_parameters = soup.body.main.find_all('div',class_="product-params")[-1].find_all('table')

            for table in product_parameters:
                temp_param = dict()
                for row in table.find_all('tr'):
                    temp_param[row.th.text.strip()] = row.td.text.strip()

                parameters[table.caption.text] = temp_param
            
            parameters['url'] = line
            product_header = soup.body.main.find_all('div',class_="product-page__header")[0].find_all('h1')[0].text
            results[product_header] = parameters
        except:
            continue


100%|████████████████████████████████████████████████████████████████████████████████| 881/881 [50:19<00:00,  3.43s/it]


In [31]:
#Запишем результат в файл json
fp = open('wildberries.json', 'a')
js = json.dumps(results)
fp.write(js)
fp.close()

In [32]:
len(results)

794

### Сбермаркет

В файле 'urls_sbmm.csv' должны быть только url без заголовка (по аналогии с ВБ)

In [23]:
results_sbmm = {}
line_num = 0
with open('urls_sbmm.csv', 'r') as file:
    for line in tqdm(file.readlines()):
        try:
            url = line

            #инициализируем словари для временного храниения информаци
            seller = dict()
            parameters = dict()

            driver.get(url=url)

            #даем странице время прогрузиться
            time.sleep(2 + 2 * random.random())

            #объект bs4 для парсинга
            soup = BeautifulSoup(driver.page_source, 'lxml')
            
            #вытаскиваем параметры из таблицы "О товаре"
            product_parameters = (soup.body.main.find_all('div',class_='pdp-specs__list')
                                  )
            #
            parameters = dict()
            for table in product_parameters[0].find_all(class_='pdp-specs__group-info'):
                #print(table)
                for row in table.find_all(class_=re.compile('^pdp-specs__item-')):
                    if row['class'][0] == 'pdp-specs__item-name':
                        param_name = row.text.strip()
                    elif row['class'][0] == 'pdp-specs__item-value':
                        param_value = row.text.strip()
                        parameters[param_name] = param_value
                        
            product_url = url
            results_sbmm[product_url] = parameters
            
        except:
            continue

100%|████████████████████████████████████████████████████████████████████████████| 2694/2694 [5:55:40<00:00,  7.92s/it]


In [24]:
len(results_sbmm)

2666

In [25]:
#Запишем результат в файл json
fp = open('sbmm.json', 'a')
js = json.dumps(results_sbmm)
fp.write(js)
fp.close()